# Reinforcement Learning 
- While teaching a helicopter how to fly, your job is to input the orientation and position of the the helicopter and output a set of numbers that correspond to where to move the control stick, to execute whatever you want.  
- This is different from supervised learning. 
- We do not know what exactly the "right answer" is. We cannot come up with a traning set of X (the position and orientation, 2 features) and the output y (the right actions to take). There can be multiple right answers here, and it's really tough to come up with a comprehensive training set. 
- Using a training set won't even work. This is something different.  
- We'll come up with a reward signmal which will measure how well the agent/helicopter is doing and it will be the job of the learning algorithm to take reward functions as inputs and try to fly well.  

- Another example would be to teach an agent to play chess. 
- We cannot come up with a training set with X's being the board positions and the y's being the optimal move to make. This is because we cannot have one optimal move at a position! There are multiple scenarios! The optimal move depends on the scenario and there might be multiple optimal moves.  
- We can give the agent a positive reward when it wins a game, and negative reward when it loses a game. And hopefully, it'll understand how to play the game eventually. 

__ Why is Reinforcement Learning harder than Supervised Learning? __ 
- This is not a one shot decision making process. 
- In Supervised Learning, you make a prediciton and then you're done. The person either had cancer or does not. That's the end of the process.
- In Reinforcement Learning, you have to make actions over time (sequential decision making).   
- Suppose your agent plays a game of chess and losses on the 60th move. Now, before the 60th move it did not get any feedback (reward). On the 60th move it got a negative reward. This makes it hard for the agent to learn as you do not know where you went wrong. This is known as the _Credit Assignment Problem_.   Suppose you went wrong on the 23rd move, and then fixes that on the 50th move, and then blunders on the 60th move... Which move do you assign blame to? 

- Reinforcement Learning problems model the world using the __MDP (Markov Decision Process)__.  
- An MDP is a 5 tuple: 
    - $S$: set of states, eg:set of all possible positions and orientations of the helicopter. 
    - $A$: set of all actions. eg: all positions we can put our control stick into. 
    - $P_{SA}$: state transition distributions. For each state and each action, this is a probability distribution. 
        - $\sum_{S'}P_{SA}(S') = 1$ The sum of all the probabilities of state you may end up in, $S'$, after taking actin $A$ with current state $S$ is $1$. 
        - $P_{SA}(S') \geq 0$ : The probability of ending up in state $S'$ after taking action $A$ in state $S$ is always $geq$ 0. In other words, there is always a possibility take you are safe at one moment, flying $S$ and then one action $A$, leads to a new state $S'$ of crash :p. 
        -  State Transition Distributions/State Transition Probabilities work as follows: 
            - $P_{SA}$ gives me the probability distribution over what state will i transition to if i take the action $A$, in the state $S$. 
    - $\gamma$ is a number known as a discount factor. It is always between 0 and 1 ($0 \leq \gamma < 1$). 
    - $R$ is our reward function. $R: S -> R$ The reward function maps from a set of states to the real numbers and it can be positive or negatve. 
    

- __Example:__  
<img src = "images/mdp.png"> 
    - $S$: 11 states 
    - $A$: actions, {N, E, W, S}
    - $P_{SA}$
        - $P_{(3,1),N}(3,2) = 0.8$  
        - $P_{(3,1),N}(4,1) = 0.1$
        - $P_{(3,1),N}(2,1) = 0.1$ 
        - $P_{(3,1),N}(3,3) = 0$ 
    - $R$:
        - $R((4,3)) = +1$
        - $R((4,2)) = -1$ 
        - $R((S)) = -0.02$ for all other states. (charges the robot for just wandering around (battery consumption).   
- In this specific example, we are assuming that there are no states once we reach $(4,3)$ or $(4,3)$. There are no more rewards after that. 
- There are many ways to model that. One way is to think of a $12^{th}$ state called the $0$ cost absorbing state. So once you get to the $+1$ or $-1$, you then transition with probability $1$ to this $12^{th}$ state which has no more rewards. 

__ How does it work? __
- You start with being in some state $S_0$.  
- You then take some actiona $A_0$. 
- You get to $S_1$ drawn randomly from $P_{S_0A_0}$ distribution. 
- Choose another action $A_1$.  
- You get to $S_2$ drawn randomly from $P_{S_1A_1}$ distribution.   
- So at the end of it you get a sequence of states ($S_0, S_1, S_2, ..... $) 
- And to evaluate how well we did, we will add up all the rewards the robot obtained at each state: 
    - $ R(S_0) + \gamma.R(S_1) + \gamma^2.R(S_2) + ..... $ 
- This is known as the _total payoff_ for the sequnce of states your robot visited.  
- $\gamma$'s effect is that it reduces the reward or _"discounts"_ the reward as steps increase. The reward you get at state 1 is bigger than reward at state 2, which is bigger than reward at state 3...... 
    - A dollar today is worth a little more than a dollar tomorrow. 
    - Conversely, having to pay out a dollar tomorrow is better than having to payout a dollor today.  
    - Basically, $\gamma$ weighs wins and losses in the immediate future more than wins and losses in the distant future.  

The goal of the reinforcement learning algorithm is to choose actions over time to try to maximise the expected value of the _total payoff_.  
* $ R(S_0) + \gamma.R(S_1) + \gamma^2.R(S_2) + ..... $   
Concretely, what we will try to compute is try to make our reinforcement learning algorithms compute a _policy_ $\pi: S -> A$. A _policy_ just a function mapping from a state to an action. So a _policy_ basically will tell us, "given a state, which action should be taken".    
So for the above example, here is the _optimal policy_:  

<img src = "images/policy.png">    


This is the _optimal policy_  in the sense that when you implement this policy, you will get the maximum _total payoff_ (total sum of discounted rewards) possible.    

Executing the _policy_ means taking the actions according to the _policy_.    

Notice $(3,1)$ goes to the west and not the north, which is the shorter path. This is because if it goes north to $3,2$,  there is a $0.1$ possibility that you end up at $4,2$ which gives a reward of $-1$.   
Therefore, it is better tot take ther longer and safer route rather than the shorter and dangerous.   


### Defining $V^{\pi} , V^*$ and $\pi^*$   
- It'll be the consequence of these definintions that $\pi^*$ will be the optimal policy.    

### $V^{\pi}$:  
- For any policy $\pi$, we are going to define a value function $V^{\pi}: S -> R$ s.t $V^{\pi}(S)$ is the expected total payoff starting in state $S$ and excuting policy $\pi$.     
## $V^{\pi}(S) = E[R(S_0) + \gamma.R(S_1) + ... | \pi , S_0 = S)] $  

__Concrete Example: __    

<img src = "images/V_pi_example.png" >   

The above example shows a policy on the left and a value function on the right. The value function tells us the _total expected payoff_ when we start from a particular _state_.  
So, if we start from $(1,3)$ , we get a positive payoff of $.52$.


- $\mathbf{V^{\pi}(S) = E[R(S_0) + \gamma(R(S_1) + \gamma(R(S_2)) +\gamma(R(S_3)) + ... )| \pi , S_0 = S)]} $    
    - ### $R(S_0)$: 
        - is sometimes known as the immediate reward. It is the reward you get for starting at this state. 
    - The consequent steps are known as _future rewards_.  
    - ### $R(S_1) + \gamma(R(S_2)) +\gamma(R(S_3)) + ... )$ is $V^{\pi}(S_1)$: 
        - the steps you would go through if you had started with $S_1$.   
    - $\mathbf{V^{\pi}(S_0) = E[R(S_0) + \gamma(R(S_1) + \gamma(R(S_2)) +\gamma(R(S_3)) + ... )| \pi , S_0 = S)] }$  
    - $\mathbf{V^{\pi}(S_0) = R(S_0) + \gamma[\sum_{S_1}P_{S_0\pi(S_0)}(S_1)V^{\pi}(S_1)]}$

### Bellman's Equations
## $V^{\pi}(S) = R(S) + \gamma\sum_{S'}P_{S\pi(S)}(S')V^{\pi}(S')$ 
- Why are the probabilities of transitioning to as tate written as $\mathbf{P_{S \pi(S)}(S')}$ and not $\mathbf{P_{S A}(S')}$ ? 
    - Because, here we are following a policy. The action we will take in state $S$ will be $\pi(S)$. 
- So what this equation is saying is the same thing as the above equations. You start with your first state $S_0$ / $S$, here. Then, further, your steps are basically $\gamma$ times the value function of the next state you enter in$S'$.  Since $S'$(the next state) is ranomly distributed, you first need to account for its probability, and then multiply it by the value function of $S'$.   

- Bellman's Equations gives you a way to solve for the value function for a policy.  
- Suppose you are given a fixed policy. How to you solve for $V^{\pi}?$.  
- You can write down Bellman's equation for every state in your MDP and this imposes a set of linear constraints on what the value function could be. 

__Concretely: __  
## $V^{\pi}(S) = R(S) + \gamma\sum_{S'}P_{S\pi(S)}(S')V^{\pi}(S')$   
Let's say $\pi(3,1) = N$   
$V^{\pi}(3,1) = R(3,1) + \gamma[0.8$$\mathbf{V^{\pi}(3,2)}$ $ + 0.1$$\mathbf{V^{\pi}(4,1)}$ + 0.1$\mathbf{V^{\pi}(2,1)}] $  
The above is the bellman equation for the $(3,1)$ state. It is the immediate reward , $R(3,1)$, plus gamma times the sumation of the all the probable (value functions, states) it could go in. $[0.8$_$\mathbf{V^{\pi}(3,2)}$_ $ + 0.1$$\mathbf{V^{\pi}(4,1)}$ + 0.1$\mathbf{V^{\pi}(2,1)}] $  
- Notice, the value functions in __bold__ are the unknowns here. If we write bellman's equation for each of the states, we can solve the linear equations to get a value function for a particular state. 
- This was the value function for a specific given policy and how to solve for it.  

## Optimal Value Function: $ V^*(S) $ 
### $V^*(S) = max_{\pi} V^\pi(S)$ 
- For any given state, $S$, the optimal value function, suppose i take a $max$ over all possible policies $\pi$, what is the best possible expected sum of discounted rewards that i can get. 
- So basically, given a state, what is the best policy i can implement (that will of course, give me the best _total payoff_).  

### Bellman's Eqautions for $V^*(S)$:  
## $V^*(S) = R(S) + \mathbf{max_A} \gamma\sum_{S'}P_{SA}(S')V^*(S')$  
- My optimal expected total payoff is my immediate reward plus the best action i can choose of my expected future payoff.    
- DOUBT: The max should be for every value function, not just for the second value function: 
    - Basically what this is saying is that, you are following the optimal policy, which will give you the optimal reward once you reach the next state. 
    - You just need to make sure that you execute the best action possible such that it maximises the future rewards. 
    - Concretely, you are in some state $s$, you take some action $a$ and then, depedning on that action, you expected total payoff will be calculated using : $\mathbf{\gamma\sum_{S'}P_{SA}(S')V^*(S')}$. 
    - So, the only thing you need to be careful about is, taking the right action. 
    - Once you take the right action, you canbe sure that the optimal policy will take care of the rest. 
## ${\pi}^*(S) = \mathbf{argmax_A}  \sum_{S'}P_{SA}(S')V^*(S')$   
- When you are in some state $s$, you are definitely going to get an immediate reward for taking any action in state $s$. 
- Now, what will differentiate that reward among different actions is the consequent rewards from states you might end up in. 
- You want to choose the action that maximises the total expected reward from the current state. 
- Start by taking one action, $a$ from state $s$. Calculate the $probability*value$ $function$ pairs for all the states that you might end up in after taking action $a$. Sum them. Do this for all the actions possible. Choose the action that reaps the best reward from state $s$. 
- __Tells you what is the best action to take in each state by taking the argmax.__

- Notice, if we can compute $V^*(S)$, _the optimal value function_, we can input it into the equation for $\pi^*(S)$ to get the _optimal policy_  
- So, the strategy for computing the _optimal policy_ would be to compute $V*(S)$, which the maximum return you can get starting from state $s$ and following policy $*$. 
- But the definition of $V^*(S) = max_{\pi} V^\pi(S)$  does not really help us to compute it cause there are an exponentially large number of policies. This is because if you 4 actions and 11 states, then the number of policies is $4^{11}$.   
- We know how to compute $V^{\pi}$: 
    - For any policy $\pi$, we are going to define a value function $V^{\pi}: S -> R$ s.t $V^{\pi}(S)$ is the expected total payoff starting in state $S$ and excuting policy $\pi$.   
- by solving the system of linear equations. 

### Value Iteration:  
- Initialise $V(S) = 0$ $\forall S$: Create an array of $n$ elements where $n$ is the the number of states. 
- For every S, update:  
    - $V(S) := R(S) + max_A \gamma \sum_S' P_{SA}(S')V(S')$  
- This will make $V(S) -> V^*(S)$ 
- DOUBT: Which policy are we following here while computing RHS? Or is this just random?   

- Synchronous Update: Symultaneously updateing all $V(S)$. 
- Asynchronus Update: Update $V(S)$ one by one.  

- When you compute the value function using the value iteration and $\gamma = 0.99$.  
- The number you get for $V^*$ are as follows: 

<img src="images/v_star.png">

- Again, this is the maximum reward you can get from starting from state $s$ and following the optimal policy. 
- Now, how do we get the optimal policy once we compute the optimal value function? 
- We use the following equation: 
    - ${\pi}^*(S) = \mathbf{argmax_A}  \sum_{S'}P_{SA}(S')V^*(S')$   
- which gives us the optimal policy: 
<img src = "images/policy.png">   

### Policy Iteration: 
- Initialise the policy $\pi$ randomly. 
- Repeat: 
    - Let $V := V^{\pi}(S)$
        - $\mathbf{V^{\pi}(S) = R(S) + \gamma\sum_{S'}P_{S\pi(S)}(S')V^{\pi}(S')}$  
    - Let $\pi(s) := \mathbf{argmax_A}  \sum_{S'}P_{SA}(S')V^{\pi}(S')$   

- DOUBT: How is it getting better at every loop? 
    - Every time you go through the loop, you calculate the value function for all the states using current policy. 
    - Then you calculate a new policy, that takes the best out of what was given. 
    - So this kinda helps me understand how this is getting better. 

### What if you do not know the State Transition Probability?   
- Let's say you are trying to fly a helicopter. In this case you will not be able to have a STD cause helicopter dynamics are quite noisy. 
- One standard thing to do is to estimate the state transition probability from data.  
- # $ P_{SA}(S') = \frac{times\ took\ action\ a\ action \ a\ in\ state\ s\ ,\ got\ state\ s'\ }{ times\ took\ action\ a\ in\ state\ s}$  
- or $ \frac{1}{|S|} if \frac{0}{0}$ , like a uniform distribution over all states if you have never tried action a in state s.   
- Putting it together: 
    - Repeat: 
        - Take actions using some policy $\pi$ to get experience in $MDP$. 
        - Update estimates of your state transition probability based on the experience you gained. 
        - Solve Bellman's equation using Value Iteration to get an estimate for $V^*$. 
        - Update policy  $\pi(s) := \mathbf{argmax_A}  \sum_{S'}P_{SA}(S')V^*(S')$   
        